In [1]:
import torch
import pprint
import evaluate
import numpy as np

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)

C:\Users\shiva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('./final_data.csv')
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,preprocessed_text,Id.1,Category
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,summarize: i have bought several of the vitali...,1.0,Pet Food
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,summarize: product arrived labeled as jumbo sa...,2.0,Snacks
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,summarize: this is a confection that has been ...,3.0,Confectionery
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,summarize: if you are looking for the secret i...,4.0,Beverages
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,summarize: great taffy at a great price. there...,5.0,Pet Food
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,B000P41A28,A3A63RACXR1XIL,"A. Boodhoo ""deaddodo""",10,15,1,1204502400,constipation,we switched from the advance similac to the or...,summarize: we switched from the advance simila...,NaN,product review
9996,9997,B000P41A28,A5VVRGL8JA7R,Adam,2,3,5,1306368000,Constipation Not A Problem if...,"Like the bad reviews say, the organic formula ...","summarize: like the bad reviews say, the organ...",NaN,product review
9997,9998,B000P41A28,A2TGDTJ8YCU6PD,geena77,0,0,5,1347494400,Love this formula!,I wanted to solely breastfeed but was unable t...,summarize: i wanted to solely breastfeed but w...,NaN,product review
9998,9999,B000P41A28,AUV4GIZZE693O,"Susan Coe ""sueysis""",1,2,5,1203638400,very convenient,i love the fact that i can get this delieved t...,summarize: i love the fact that i can get this...,NaN,product review


In [3]:
data = df[:1000]
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data, test_size=0.2, random_state=42)

In [4]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
tokenizer = T5Tokenizer.from_pretrained("./Finetuned_T5/tokenizer")
model = T5ForConditionalGeneration.from_pretrained("./Finetuned_T5/model-2").to(device)

In [12]:
def summarize_text_finetuned(text, max_input_length=512, max_output_length=10):
    """
    Summarizes input text using a T5 model.
    """
    # Add the "summarize:" prefix for T5
    input_text = text.strip()
    
    # Tokenize the input text
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=max_input_length).to(device)
    
    # Generate the summary
    outputs = model.generate(inputs, max_length=max_output_length, min_length=2, length_penalty=2.0, num_beams=4)
    
    # Decode the summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [ ]:
dataset_test["generated_summary"] = dataset_test["preprocessed_text"].apply(summarize_text_finetuned)

In [8]:
rouge = evaluate.load("rouge")

Using the latest cached version of the module from C:\Users\shiva\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--rouge\b01e0accf3bd6dd24839b769a5fda24e14995071570870922c71970b3a6ed886 (last modified on Sat Dec 14 15:57:24 2024) since it couldn't be found locally at evaluate-metric--rouge, or remotely on the Hugging Face Hub.


In [ ]:
results = rouge.compute(
    predictions=dataset_test["generated_summary"].tolist(),
    references=dataset_test["Summary"].tolist()
)
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

rouge1: 0.1769
rouge2: 0.0747
rougeL: 0.1677
rougeLsum: 0.1686


In [11]:
dataset_test["generated_summary"]

521                                      cheesy & cheesy
737    I love the ready to spread frosting, especiall...
740      I don't think it would be the right consistency
660     Tastes better than other low glycemic sweeteners
411                                 I love this product.
                             ...                        
408             a great fish - perfect for all purposes!
332    Great for school snacks, lunch boxes, and scho...
208    Not a dragon breath person, but a good breath ...
613    Good for a Natural Obsession - Good for a Natural
78              Great taste, but the berries had melted.
Name: generated_summary, Length: 200, dtype: object